In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score

In [10]:
import pandas as pd

# Read CSV and preprocess the data
future = pd.read_csv("./data/test_future.csv")
future = future.sort_values(["종목코드", "일자"]).reset_index(drop=True)
future = future.drop(["거래량", "시가", "고가", "저가"], axis=1)

def calculate_return(df):
    return df.groupby('종목코드')['종가'].pct_change()

def calculate_std(df):
    return df.groupby('종목코드')['수익률'].std()

def calculate_final_return(df):
    first_day_closing = df.groupby('종목코드')['종가'].first()
    last_day_closing = df.groupby('종목코드')['종가'].last()
    return ((last_day_closing - first_day_closing) / first_day_closing).to_frame()

# Calculate returns and add to the DataFrame
future['수익률'] = calculate_return(future)

# Calculate standard deviation
std_df = calculate_std(future)
std_df = std_df.reset_index()

# Calculate final return and Sharpe ratio
final_return_df = calculate_final_return(future)
result = std_df.merge(final_return_df, on='종목코드')
result = result.rename(columns={"수익률" : "std", "종가" : "final_return"})
result

,종목코드,std,final_return
0,A000020,0.015429,0.119792
1,A000040,0.009978,-0.104478
2,A000050,0.006591,-0.035680
3,A000070,0.012383,0.056657
4,A000080,0.010666,-0.050218
...,...,...,...
1992,A375500,0.013871,0.012397
1993,A378850,0.021294,0.026316
1994,A383220,0.017311,0.010046
1995,A383310,0.023728,0.030547


## 투자종목 선정 (상위 200 , 하위 200)

In [11]:
xgboost = pd.read_csv("./sub/XGBoost3_-파라미터서칭.csv")

xgboost = xgboost.sort_values("순위", ascending=False)

xgboost_short = xgboost[:200]
xgboost_long = xgboost[-200:]

xgboost_short = xgboost_short['종목코드'].to_list()
xgboost_long = xgboost_long['종목코드'].to_list()

In [15]:
invest = result.sort_values('final_return')
short = invest[invest['종목코드'].isin(xgboost_short)]
long = invest[invest['종목코드'].isin(xgboost_long)]

# 총자산 최종수익률
asset_return = (short['final_return'].sum() * -1 + long['final_return'].sum()) / 400

# 연율화된 총자산 최종 수익률
annualised_return = asset_return * 250 / 15
annualised_return

-0.24593046067737132

In [18]:
allPortfolio = short.merge(long, how='outer')
allPortfolio

df3 = allPortfolio[['return1', 'return2', 'return3', 'return4', 'return5', 'return6', 'return7', 'return8', 'return9', 'return10', 'return11', 'return12', 'return13', 'return14']] * 250

,종목코드,std,final_return
0,A052020,0.038940,-0.282185
1,A126600,0.019892,-0.165536
2,A082800,0.016664,-0.160494
3,A016790,0.068736,-0.148148
4,A043590,0.023678,-0.147193
...,...,...,...
395,A373200,0.085640,0.463626
396,A018000,0.092672,0.616933
397,A338220,0.076066,0.664141
398,A357550,0.099439,0.802713


## 샤프 계산

In [ ]:
# n일 째 수익률의 평균 계산
mean_return = df3.mean(axis=0)

# 전체 수익률의 평균 계산
total_mean = df3.mean(axis=1).mean(axis=0)

# N번째 매매일의 일간 수익률 평균 수익률과 일간 수익률의 평균의 차이의 제곱의 합을 계산
diff_sum = (mean_return - total_mean) ** 2

# 위 결과를 n - 2로 나누고, 그 제곱근을 구함
sharpe = np.sqrt(diff_sum / (15 - 2))
sharpe.to_frame().sum()

In [ ]:
#RMSE는 예측값과 실제 값 간의 차이를 제곱하여 평균을 낸 후, 그 결과의 제곱근을 구하는 방식으로 계산됩니다. RMSE 값이 작을수록 예측이 정확하다는 것을 의미합니다.
#평균 절대 오차(Mean Absolute Error, MAE) 또한 유사성을 판단하는 데 사용할 수 있습니다. MAE는 예측값과 실제 값의 차이의 절대값을 평균한 값입니다.
#결정 계수(R-squared, R^2) 는 예측값이 실제 값에 얼마나 가까운지를 나타내는 통계적 척도입니다. R^2 값이 1에 가까울수록 예측이 실제 값을 잘 설명하고 있다는 것을 의미합니다.

# 실제 종가 데이터
file_dir = "C:\Users\82107\OneDrive - 경희대학교\빅데이터응용학과\KRX 박재균/KRX 평가용.csv" # 여기에 csv 경로 입력
df_actual = pd.read_csv(file_dir)
df_actual['일자'] = pd.to_datetime(df_actual['일자'].astype(str))
df_actual_20230621 = df_actual[df_actual['일자'] == '2023-06-21']

# 예측 종가 데이터
df_predicted = pd.read_csv('/mnt/data/predicted_prices.csv')
df_predicted['일자'] = pd.to_datetime(df_predicted['일자'].astype(str))
df_predicted_20230621 = df_predicted[df_predicted['일자'] == '2023-06-21']

# RMSE 계산
rmse = np.sqrt(mean_squared_error(df_actual_20230621['종가'], df_predicted_20230621['종가']))

# MAE 계산
mae = mean_absolute_error(df_actual_20230621['종가'], df_predicted_20230621['종가'])

# R^2 계산
r2 = r2_score(df_actual_20230621['종가'], df_predicted_20230621['종가'])


print(rmse, mae, r2)